# SYDE 556/750 --- Assignment 4
**Student ID: 20823934**

*Note:* Please include your numerical student ID only, do *not* include your name.

*Note:* Refer to the [PDF](https://github.com/celiasmith/syde556-f22/raw/master/assignments/assignment_04/syde556_assignment_04.pdf) for the full instructions (including some hints), this notebook contains abbreviated instructions only. Cells you need to fill out are marked with a "writing hand" symbol. Of course, you can add new cells in between the instructions, but please leave the instructions intact to facilitate marking.

In [ ]:
# Import numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt

# Import nengo and some helper functions for Q1
import nengo
from nengo.utils.ensemble import tuning_curves
from nengo.utils.connection import eval_point_decoding

# Fix the numpy random seed for reproducible results
np.random.seed(18945)

# Some formating options
%config InlineBackend.figure_formats = ['svg']

In [ ]:
import inspect
sig = inspect.signature(nengo.Ensemble)
for param in sig.parameters.values():
    print(param)

# 1. Building an ensemble of neurons

**a) Tuning curves.** Plot the population tuning curves. Plot the representation accuracy plot ($x - \hat{x}$). Compute and report the RMSE.

In [ ]:
def q1_ensemble(tau_rc=0.02, tau_ref=0.002, radius=1, seed=0):
    # Generate the 1D network
    model = nengo.Network()
    with model:
        a = nengo.Ensemble(n_neurons=100,                                   # 100 neurons
                           dimensions=1,                                    # 1-dimensional space
                           intercepts=nengo.dists.Uniform(-1, 1),           # x-intercepts between -1 and 1
                           max_rates=nengo.dists.Uniform(100, 200),         # max firing rates between 100 and 200 Hz
                           neuron_type=nengo.neurons.LIF(tau_rc=tau_rc, 
                                                         tau_ref=tau_ref),  # LIF neurons
                           radius=radius,                                   # radius
                           seed=seed)                                       # random seed
        
        a_a = nengo.Connection(pre=a, post=a)

    # Simulate the tuning curves
    with nengo.Simulator(model) as sim:
        tuning_x, activities = tuning_curves(a, sim)

    # Calculate the representation accuracy and sort them by eval_points
    eval_points, targets, decoded = eval_point_decoding(a_a, sim, eval_points=None)
    sorted_zipped = sorted(zip(eval_points, targets, decoded), key=lambda x: x[0])
    eval_points, targets, decoded = zip(*sorted_zipped)
    eval_points = np.array(eval_points)
    targets = np.array(targets)
    decoded = np.array(decoded)

    # Compute the RMSE
    rmse = np.sqrt(np.mean((targets - decoded)**2))

    return tuning_x, activities, targets, decoded, rmse

In [ ]:
def q1a():
    tuning_x, activities, targets, decoded, rmse = q1_ensemble()

    # Plot the tuning curves
    plt.figure()
    plt.plot(tuning_x, activities)
    plt.title("Tuning curves for 100 neurons")
    plt.xlabel("Input x")
    plt.ylabel("Firing rate (Hz)")
    plt.show()

    # Plot the representation accuracy
    plt.figure()
    plt.plot(targets, targets - decoded)
    plt.title("Representation accuracy (x - x_hat)")
    plt.xlabel("Input x")
    plt.ylabel("x - x_hat")
    plt.show()
    
    # Report the RMSE
    print("RMSE: %0.3f" % rmse)

q1a()

**b) RMSE and radius.** Compute the RMSE for (at least) the four different radii $0.5$, $1$, $2$, and $4$. Plot your results. Make sure your neurons have the same (relative, i.e., scaled by the radius) $x$-intercepts and maximum rates across all experiments.

In [ ]:
def q1b():
    # Calulate RMSE for different radii
    radii = [2 ** power for power in range(-1, 3)]
    rmses = []
    for radius in radii:
        tuning_x, activities, targets, decoded, rmse = q1_ensemble(radius=radius)
        rmses.append(rmse)

    # Plot RMSE vs. radius
    plt.figure()
    plt.plot(radii, rmses)
    plt.title("RMSE vs. radius")
    plt.xlabel("Radius")
    plt.ylabel("RMSE")
    plt.show()

    # Calculate y=mx+b
    m = (rmses[-1] - rmses[0]) / (radii[-1] - radii[0])
    b = rmses[0] - m * radii[0]
    print(f"RMSE = {m:.3f} * radius + {b:.3f}")
    print(f"Slope: {m:.6f}, RMSE at radius 1: {rmses[1]:.6f}")
    
q1b()

**c) Discussion.** What mathematical relationship between the radius and the RMSE do you observe (write down an equation)? Explain why this is the case.

For an ensemble of neurons with different radii but the same seed, the mathematical equation is $RMSE = m * R$ where slope m is the RMSE at $R=1$. This is the case because
increasing the radius stretches the tuning curves horizontally (and decreasing the radius compresses them). As a result, the neurons are less sensitive to input 
$x$, and the same change in $x$ leads to a smaller change in firing rate. As the radius shrinks to 0, the neurons become very sensitive, and the error also approaches 0.

**d) RMSE and refractory period.** What happens to the RMSE and the tuning curves as $\tau_\mathrm{ref}$ changes between $1$ and $5\,\mathrm{ms}$? Plot the tuning curves for at least four different $\tau_\mathrm{ref}$ and produce a plot showing the RMSE over $\tau_\mathrm{ref}$. Again, make sure to use the same neuron ensemble parameters in all your trials.

In [ ]:
def q1d():
    # Calulate RMSE for different refractory periods
    tau_refs = np.linspace(0.001, 0.005, 4) # 1 ms to 5 ms
    tuning_xs = []
    activitiess = []
    rmses = []
    for tau_ref in tau_refs:
        tuning_x, activities, targets, decoded, rmse = q1_ensemble(tau_ref=tau_ref)
        tuning_xs.append(tuning_x)
        activitiess.append(activities)
        rmses.append(rmse)

    # Plot the tuning curves
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    for i in range(4):
        axs[i].plot(tuning_xs[i], activitiess[i])
        axs[i].set_title(f"Tuning curves for tau_ref = {tau_refs[i]:.4f}")
    fig.supxlabel("Input x")
    fig.supylabel("Firing rate (Hz)")
    plt.tight_layout()
    plt.show()

    # Plot RMSE vs. tau_ref
    plt.figure()
    plt.plot(tau_refs, rmses)
    plt.title("RMSE vs. tau_ref")
    plt.xlabel("tau_ref")
    plt.ylabel("RMSE")
    plt.show()
    
q1d()

**e) RMSE and membrane time constant.** What happens to the RMSE and the tuning curves as $\tau_\mathrm{RC}$ changes between $10$ and $100\,\mathrm{ms}$? Plot the tuning curves for at least four different $\tau_\mathrm{RC}$ and produce a plot showing the RMSE over $\tau_\mathrm{RC}$.  Again, make sure to use the same neuron ensemble parameters in all your trials.

In [ ]:
def q1e():
    # Calulate RMSE for different membrane time constants
    tau_rcs = np.linspace(0.01, 0.1, 4) # 10 ms to 100 ms
    tuning_xs = []
    activitiess = []
    rmses = []
    for tau_rc in tau_rcs:
        tuning_x, activities, targets, decoded, rmse = q1_ensemble(tau_rc=tau_rc)
        tuning_xs.append(tuning_x)
        activitiess.append(activities)
        rmses.append(rmse)

    # Plot the tuning curves
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    for i in range(4):
        axs[i].plot(tuning_xs[i], activitiess[i])
        axs[i].set_title(f"Tuning curves for tau_ref = {tau_rcs[i]:.4f}")
    fig.supxlabel("Input x")
    fig.supylabel("Firing rate (Hz)")
    plt.tight_layout()
    plt.show()

    # Plot RMSE vs. tau_rc
    plt.figure()
    plt.plot(tau_rcs, rmses)
    plt.title("RMSE vs. tau_ref")
    plt.xlabel("tau_ref")
    plt.ylabel("RMSE")
    plt.show()
    
q1e()

**f) Discussion.** Discuss the last two results. Describe what happens to the tuning curves as $\tau_\mathrm{ref}$ and $\tau_\mathrm{RC}$ change (you do not need to come up with a mathematical relationship here). Explain why the change in tuning curve shape influences the RMSE in the way you observe.

As $\tau_{ref}$ increases from 1 ms to 5 ms, the top of the tuning curves become more rounded. However, as $\tau_{rc}$ increases from 10 ms to 100 ms, the bottom of the tuning curves become more straight. The RMSE decreases as the tuning curves become less rounded / more straight because the identity relationship is linear ($x = \hat{x}$), so linear tuning curves are better able to capture that information.

# 2. Connecting neurons

**a) Computing the identity function.** Show the input value and the decoded values from the two  ensembles in three separate plots. Run the simulation for $0.5\,\mathrm{s}$.

In [ ]:
def q2_ensemble(tau_rc=0.02, tau_ref=0.002, radius=1, func=None, sim_time=0.5, seed=0):
    # Generate the 1D network
    model = nengo.Network()
    with model:
        a = nengo.Ensemble(n_neurons=100,                                   # 100 neurons
                           dimensions=1,                                    # 1-dimensional space
                           intercepts=nengo.dists.Uniform(-1, 1),           # x-intercepts between -1 and 1
                           max_rates=nengo.dists.Uniform(100, 200),         # max firing rates between 100 and 200 Hz
                           neuron_type=nengo.neurons.LIF(tau_rc=tau_rc, 
                                                         tau_ref=tau_ref),  # LIF neurons
                           radius=radius,                                   # radius
                           seed=seed)                                       # random seed
        
        b = nengo.Ensemble(n_neurons=50,                                    # 50 neurons
                           dimensions=1,                                    # Everything else is the same as above
                           intercepts=nengo.dists.Uniform(-1, 1),
                           max_rates=nengo.dists.Uniform(100, 200),
                           neuron_type=nengo.neurons.LIF(tau_rc=tau_rc, 
                                                         tau_ref=tau_ref),
                           radius=radius,
                           seed=seed)

        # Create a step-function input that is a value of 1 for 0.1 < t < 0.4 seconds, and otherwise is zero
        input = nengo.Node(lambda t: 0.0 if t < 0.1 else (1.0 if t < 0.4 else 0.0))
        
        # Connect with a post-synaptic time constant of 10ms
        nengo.Connection(pre=input, post=a, synapse=0.01)
        if func is None:
            nengo.Connection(pre=a, post=b, synapse=0.01)
        else:
            nengo.Connection(pre=a, post=b, function=func, synapse=0.01)

        # Probes use a synaptic filter of 10 ms
        in_probe = nengo.Probe(input)
        a_probe = nengo.Probe(a, synapse=0.01)
        b_probe = nengo.Probe(b, synapse=0.01)

    # Run the simulation for some time
    with nengo.Simulator(model) as sim:
        sim.run(sim_time)

    # Plot the results
    fig, axs = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
    axs[0].plot(sim.trange(), sim.data[in_probe])
    axs[0].set_title("Input")
    axs[1].plot(sim.trange(), sim.data[a_probe])
    axs[1].set_title("A")
    axs[2].plot(sim.trange(), sim.data[b_probe])
    axs[2].set_title("B")
    plt.show()

q2_ensemble()

**b) Computing an affine transformation.** Make a new version of the model where instead of computing the identity function, it computes $y(t) = 1 - 2x(t)$. Show the same graphs as in part (a).

In [ ]:
def func_q2b(x):
    return 1 - 2 * x[0]
q2_ensemble(func=func_q2b)

# 3. Dynamics

**a) Transforming the dynamical system.** Rewrite the linear dynamical system describing the integrator in terms of $\frac{\mathrm{d}\vec x(t)}{\mathrm{d}t} = \mathbf{A} \mathbf{x} + \mathbf{B} \mathbf{u}$, i.e., write down the matrices $\mathbf{A}$ and $\mathbf{B}$ (you can just use the equations from class, you do not have to re-derive the equations) What are the matrices $\mathbf{A}'$ and $\mathbf{B}'$ we have to use when implementing this system using the recurrent connection post-synaptic filter?

From lecture notes, an integrator is defined as the dynamical system:
$$\frac{\mathrm{d}\vec x(t)}{\mathrm{d}t} = \mathbf{u}$$

Therefore, the matrices are:

$$
\begin{align}
    \mathbf{A} &= \mathbf{0} \\
    \mathbf{B} &= \mathbf{I}
\end{align}
$$

The neural implementation of LTI systems has the transformation:

$$
\begin{align}
    \mathbf{A}^\prime &= \tau\mathbf{A} + \mathbf{I} \\
    \mathbf{B}^\prime &= \tau\mathbf{B}
\end{align}
$$

So our matrices become:

$$
\begin{align}
    \mathbf{A}^\prime &= \tau\mathbf{0} + \mathbf{I} = \mathbf{I} \\
    \mathbf{B}^\prime &= \tau\mathbf{I}
\end{align}
$$

Using the longer time constant for $\tau$ (50ms), the 1-dimensional transforms are:

$$
\begin{align}
    a^\prime &= 1 \\
    b^\prime &= 0.05
\end{align}
$$

**b) Integrator using spiking neurons.**  Show the input, the ideal integral, and the value represented by the ensemble when the input is a value of $0.9$ from $t=0.04$ to $t=1.0$ (and $0$ for other times). Run the simulation for $1.5\,\mathrm{s}$.

In [ ]:
def q3_ensemble(input_func, neuron_type=nengo.neurons.LIF, tau_rc=0.02, tau_ref=0.002, radius=1, sim_time=0.5, seed=0):
    # Generate the 1D network
    model = nengo.Network()
    with model:
        a = nengo.Ensemble(n_neurons=200,                               # 200 neurons
                           dimensions=1,                                # Everything else is the same
                           intercepts=nengo.dists.Uniform(-1, 1),
                           max_rates=nengo.dists.Uniform(100, 200),
                           neuron_type=neuron_type(tau_rc=tau_rc,       # LIF or LIFRate neurons
                                                   tau_ref=tau_ref),
                           radius=radius,
                           seed=seed)
        
        input = nengo.Node(input_func)
        
        # Build the connections
        nengo.Connection(pre=a, post=a, transform=[[1]], synapse=0.05)          # Recurrent connection (transform = 1, τ = 50ms)
        nengo.Connection(pre=input, post=a, transform=[[0.05]], synapse=0.005)  # Input connection (transform = 0.05, τ = 5ms)

        # Probes use a synaptic filter of 10 ms
        in_probe = nengo.Probe(input)
        a_probe = nengo.Probe(a, synapse=0.01)

    # Run the simulation for some time
    with nengo.Simulator(model) as sim:
        sim.run(sim_time)

    # Calculate the ideal integral
    ideal_integral = np.cumsum(sim.data[in_probe]) * 0.001

    # Plot the results
    fig, axs = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
    axs[0].plot(sim.trange(), sim.data[in_probe])   # Input
    axs[0].set_title("Input")
    axs[0].set_xlabel("Time (s)")
    axs[0].set_ylabel("Value")

    axs[1].plot(sim.trange(), sim.data[a_probe], label="Simulated")
    axs[1].plot(sim.trange(), ideal_integral, label="Ideal")
    axs[1].legend()
    axs[1].set_title("Output")
    axs[1].set_xlabel("Time (s)")
    plt.tight_layout()
    plt.show()

In [ ]:
# 0.9 from t = 0.04 to t = 1.0 (and 0 for other times)
input_q3b = lambda t: 0.0 if t < 0.04 else (0.9 if t < 1.0 else 0.0)

# Run the simulation for 1.5s
q3_ensemble(input_func=input_q3b, sim_time=1.5, seed=0)

**c) Discussion.** What is the expected ideal result, i.e., if we just mathematically computed the integral of the input, what is the equation describing the integral? How does the simulated output compare to that ideal?

The ideal integral is:

$$
	\int_{0}^{t} u(t^\prime) \,dt^\prime = \begin{cases}
		0 & \text{if } t < 0.04 \,, \\
		0.9(t - 0.04) & \text{if } 0.04 \le t < 1 \,, \\
		0.864 & \text{if } 1 \le t \,.
	\end{cases}
$$

The simulated output mostly matchs the ideal output, although there are 3 main deviations:

1. It has a lot of high frequency noise because the neurons use the spiking LIF model. This error can be reduced by increasing the number of neurons.
2. The simulated output is a bit higher than the ideal output because the time constants don't match. When $\tau$ was changed to 50 ms for the input-ensemble connection (to match the recurrent connection), the average simulated output was much closer to the ideal.
3. The random seed determines the location/coverage of the tuning curves, which affects how well the signal can be decoded. For some seeds, the ramp part of the output is very close to the ideal, while other seeds are better at the horizontal part.

**d) Simulation using rate neurons.** Change the neural simulation to rate mode. Re-run the simulation in rate mode. Show the resulting plots.

In [ ]:
# Run the simulation for 1.5s
q3_ensemble(input_func=input_q3b, neuron_type=nengo.neurons.LIFRate, sim_time=1.5, seed=0)

**e) Discussion.** How does this compare to the result in part (b)? What deviations from the ideal do you still observe? Where do those deviations come from?

Since d) uses rate neurons instead of spiking neurons, there's no longer any high-frequency noise. However, 2 deviations still remain:

1. Due to a mismatch in time constants, the simulated outputs have a distinct LIF shape at sharp changes (when a neuron turns on). This deviation is greatly reduced when the two time constants are the same, and the simulated output looks more like a linear approximation.
2. The random seed still affects the distribution of the tuning curves, so some decodings are closer to the ideal than others. This deviation is reduced when more neurons are used (for example 1000 instead of 200).

**f) Integration of a shorter input pulse.** Returning to spiking mode, change the input to be a value of $0.9$ from $t=0.04$ to $0.16$. Show the same plots as before (the input, the ideal, and the value represented by the ensemble over $1.5\,\mathrm{s}$).

In [ ]:
# 0.9 from t = 0.04 to t = 0.16 (and 0 for other times)
input_q3f = lambda t: 0.0 if t < 0.04 else (0.9 if t < 0.16 else 0.0)

# Run the simulation for 1.5s
q3_ensemble(input_func=input_q3f, sim_time=1.5, seed=0)

**g) Discussion.** How does this compare to (b)? What is the ideal equation? Does it work as intended? If not, why is it better or worse?

f) looks similar to b) in that they have similar sources of error. Both simulated outputs go above the ideal output at the start, and stay above it at a constant distance for the remainder of the output. However, f) seems to have slightly higher amplitude noise during the constant portion.

The ideal integral becomes:

$$
	\int_{0}^{t} u(t^\prime) \,dt^\prime = \begin{cases}
		0 & \text{if } t < 0.04 \,, \\
		0.9(t - 0.04) & \text{if } 0.04 \le t < 0.16 \,, \\
		0.108 & \text{if } 0.16 \le t \,.
	\end{cases}
$$

It does work as intended as it's able to represent the ideal integral to some degree. However, the noise is relatively large in f) compared to b) because the range of f) is smaller. Since b) makes better use of the whole range (-1, 1), it appears to have a better signal-to-noise ratio.

**h) Input ramp.** Change the input to a ramp input from $0$ to $0.9$ from $t=0$ to $t=0.45$ (and $0$ for $t>0.45$). Show the same plots as in the previous parts of this question.

In [ ]:
# 0.9 from t = 0.04 to t = 0.16 (and 0 for other times)
input_q3h = lambda t: 0.0 if t < 0 else (2*t if t < 0.45 else 0.0)

# Run the simulation for 1.5s
q3_ensemble(input_func=input_q3h, sim_time=1.5, seed=0)

**i) Discussion.** What does the ensemble end up representing, and why? What is the (ideal) equation for the curve traced out by the ensemble?

The ensemble ends up representing a parabolic curve followed by a constant tail because the integral of a ramp is parabolic.

The ideal equation in this case is:

$$
	\int_{0}^{t} u(t^\prime) \,dt^\prime = \begin{cases}
		0 & \text{if } t < 0 \,, \\
		t^2 & \text{if } 0 \le t < 0.45 \,, \\
		0.2025 & \text{if } 0.45 \le t \,.
	\end{cases}
$$

**j) Sinusoidal input.** Change the input to $5\sin(5t)$. Show the same plots as before.

In [ ]:
# 0.9 from t = 0.04 to t = 0.16 (and 0 for other times)
input_q3j = lambda t: 5 * np.sin(5 * t)

# Run the simulation for 1.5s
q3_ensemble(input_func=input_q3j, sim_time=1.5, seed=0)

**k) Discussion.** What should the value represented by the ensemble be? Write the equation. How well does it do? What are the differences between the model's behaviour and the expected ideal behaviour and why do these differences occur?

The value represented by the ensemble should be:

$$
	\int_{0}^{t} u(t^\prime) \,dt^\prime = \begin{cases}
		0 & \text{if } t < 0 \,, \\
		-cos(5t)+1 & \text{if } 0 \le t \,.
	\end{cases}
$$

The simulated output has a significant deviation from the ideal output when the neurons saturate at output=1, but otherwise maintains a similar simusoidal shape as the ideal curve. Since the ideal curve has a range of [0, 2], saturation causes the simulated output to stop increasing when it reaches 1, but still able to decrease down to -1. As a result, the simulated curve takes on the range [-1, 1], which is exactly what the neurons are able to represent with radius 1.

There are still 2 other sources of error as mentioned above:

1. High-frequency noise due to the spiking LIF model
2. An upward trend at the start of the output due to the time constant mismatch.

**l) 🌟 Bonus question.** Implement a nonlinear dynamical system we have not seen in class (and that is not in the book). Demonstrate that it's working as expected

✍ \<YOUR SOLUTION HERE\>

In [ ]:
# ✍ <YOUR SOLUTION HERE>